## Heart Disease Prediction using Decision Tree and Random Forest

## Project: Final Project - Decision Tree
Dataset: UCI Heart Disease (Cleveland) Dataset
Author: [Virakden - Hikaru]
Description:
- This project implements a Decision Tree classifier to predict the
- presence of heart disease using the classic UCI Cleveland dataset.
- The workflow includes data loading, cleaning, exploratory analysis,
- model training, evaluation, visualization, and comparison with Random Forest.
- Extra tasks: try different split criteria and discuss feature importances.